<a href="https://colab.research.google.com/github/ahcamachod/2215-langchain-automatizando-el-analisis-de-datos-con-agentes/blob/aula-2/agentes_con_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/content/datos_entregas.csv')

df.head()

,ID_pedido,años_experiencia_colaborador,calificacion_colaborador,latitud_tienda,longitud_tienda,latitud_entrega,longitud_entrega,fecha_pedido,hora_pedido,hora_retirada,clima,trafico,vehiculo,area,categoria_producto,tiempo_entrega
0,ialx566343618,8,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Soleado,Alto,Motocicleta,Urbano,Ropa,120
1,akqg208421122,10,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Lluvioso,Congestion,Scooter,Metropolitano,Electrónicos,165
2,njpu434582536,16,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Tormenta,Bajo,Motocicleta,Urbano,Deportes,130
3,rjto796129700,8,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Soleado,Medio,Motocicleta,Metropolitano,Cosméticos,105
4,zguw716275638,11,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Nublado,Alto,Scooter,Metropolitano,Juguetes,150


In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
!pip install langchain-groq -q

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name='llama3-70b-8192', temperature=0)

In [ ]:
ai_msg = llm.invoke("""
                    Tengo un dataframe llamado 'df' con las columnas 'años_experiencia_colaborador' y 'tiempo_entrega'.
                    Escribe el código Python con la biblioteca pandas para calcular la correlación entre las dos columnas.
                    Devuelve la respuesta en Markdown para el trecho de código Python únicamente.
                    """)
print(ai_msg)

content='```\ncorrelacion = df[\'años_experiencia_colaborador\'].corr(df[\'tiempo_entrega\'])\nprint("La correlación entre \'años_experiencia_colaborador\' y \'tiempo_entrega\' es:", correlacion)\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 83, 'total_tokens': 138, 'completion_time': 0.101211685, 'prompt_time': 0.015470908, 'queue_time': 0.001778412, 'total_time': 0.116682593}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--1fa8db36-c26c-48c4-83de-a262cf21a281-0' usage_metadata={'input_tokens': 83, 'output_tokens': 55, 'total_tokens': 138}


In [ ]:
print(ai_msg.content)

```
correlacion = df['años_experiencia_colaborador'].corr(df['tiempo_entrega'])
print("La correlación entre 'años_experiencia_colaborador' y 'tiempo_entrega' es:", correlacion)
```


In [ ]:
correlacion = df['años_experiencia_colaborador'].corr(df['tiempo_entrega'])
print("La correlación entre 'años_experiencia_colaborador' y 'tiempo_entrega' es:", correlacion)

La correlación entre 'años_experiencia_colaborador' y 'tiempo_entrega' es: -0.2516410755060904


PythonAstREPLTool
Lee - Evalua - imprime - itera

https://python.langchain.com/api_reference/experimental/tools/langchain_experimental.tools.python.tool.PythonAstREPLTool.html

In [ ]:
!pip install langchain-experimental -q

In [ ]:
from langchain_experimental.tools import PythonAstREPLTool

herramienta_py = PythonAstREPLTool(locals={"df":df})
herramienta_py.invoke("df['años_experiencia_colaborador'].corr(df['tiempo_entrega'])")

np.float64(-0.2516410755060904)

In [ ]:
llm_con_herramienta = llm.bind_tools([herramienta_py],tool_choice=herramienta_py.name)
respuesta = llm_con_herramienta.invoke("""
                                       Tengo un dataframe llamado 'df' con las columnas 'años_experiencia_colaborador' y 'tiempo_entrega'.
                                       Escribe el código Python con la biblioteca pandas para calcular la correlación entre las dos columnas.
                                       Devuelve la respuesta en Markdown para el trecho de código Python únicamente.
                                       """)

respuesta.tool_calls

[{'name': 'python_repl_ast',
  'args': {'query': "import pandas as pd\nprint(df[['años_experiencia_colaborador', 'tiempo_entrega']].corr().iloc[0,1])"},
  'id': 'jm9tzhbww',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=herramienta_py.name, first_tool_only=True)

cadena = llm_con_herramienta | parser

respuesta = cadena.invoke("""
              Tengo un dataframe llamado 'df' con las columnas 'años_experiencia_colaborador' y 'tiempo_entrega'.
              Escribe el código Python con la biblioteca pandas para calcular la correlación entre las dos columnas.
              Devuelve el script de Python únicamente.
              """)
respuesta["query"]

"import pandas as pd\nprint(df['años_experiencia_colaborador'].corr(df['tiempo_entrega']))"

In [ ]:
import pandas as pd; corr_coef = df['años_experiencia_colaborador'].corr(df['tiempo_entrega']); print(corr_coef)

-0.2516410755060904


In [ ]:
df.columns.to_list()

['ID_pedido',
 'años_experiencia_colaborador',
 'calificacion_colaborador',
 'latitud_tienda',
 'longitud_tienda',
 'latitud_entrega',
 'longitud_entrega',
 'fecha_pedido',
 'hora_pedido',
 'hora_retirada',
 'clima',
 'trafico',
 'vehiculo',
 'area',
 'categoria_producto',
 'tiempo_entrega']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system = f"""
          Tienes acceso a un dataframe pandas `df`.
          Aqui esta la salida del comando `df.columns.to_list()`:

          ```
          {df.columns.to_list()}
          ```

          Dada una pregunta de usuario, escribe el código Python para responderla,
          Cada comando python generado SIEMPRE debe ser separado por ';'.
          Los únicos recursos que debes utilizar son las bibliotecas incorporadas Python y pandas.
          Retorna ÚNICAMENTE el código Python.
          """

prompt = ChatPromptTemplate.from_messages([("system",system),("human","{question}")])

In [ ]:
cadena = prompt | llm_con_herramienta | parser | herramienta_py

respuesta = cadena.invoke({"question":"Cuál es la correlación entre años experiencia del colaborador y tiempo de entrega?"})

print(respuesta)

-0.25164107550609055


In [ ]:
respuesta = cadena.invoke({"question":"Calcula el promedio de tiempo de entrega para cada clima?"})

print(respuesta)

clima
Lluvioso    123.211825
Neblina     136.570833
Nublado     138.286773
Soleado     103.664453
Tormenta    123.238509
Ventoso     123.658037
Name: tiempo_entrega, dtype: float64
